In [1]:
import pandas as pd
import numpy as np

from sklearn  import base
from sklearn.model_selection import train_test_split, KFold

from termcolor import colored

In [2]:
########################### Creating Toy Training Set ##############################
train= pd.DataFrame()

tr_letters= 'A B A B A C'.split()
tr_labels= [1, 4, 2, 0, 0, 6]

train['letter']= tr_letters
train['label']= tr_labels

#Mi creo una copia
copy_train= train.copy()

####### Creating Toy Test: I put on purpose one Category seen in the Train ('A'), the other ('D') unseen ########################
test_letters= 'A D'.split()
test_labels= [10, 8]

test= pd.DataFrame()
test['letter']= test_letters
test['label']= test_labels

#Mi creo una copia
copy_test= test.copy()


train

,letter,label
0,A,1
1,B,4
2,A,2
3,B,0
4,A,0
5,C,6


In [3]:
test

,letter,label
0,A,10
1,D,8


## Step by step of basic KFold Target Encoding on Training set

In [4]:
kf = KFold(n_splits = 3, shuffle = False, random_state= 0)

name_encoded_mean= train.columns[0] + '_encoded_mean'
name_encoded_std= train.columns[0] + '_encoded_std'

column_to_encode= 'letter'
target_column= 'label'

################################### INIZIA CICLO ################################
threshold= 0
count= 1
show_steps= True

print('\033[1m\tINITIAL TRAIN \n {} \n \033[0m'.format(train) )
print(colored('\033[1m Threshold scelta: {}'.format(threshold), 'red') )
for tr_ind, val_ind in kf.split(train):
    
    if show_steps== True:
        continua= str(input('Continue loop? y/n \n'))
        if continua.lower()[0]!= 'y':
            print('Stoppo il ciclo')
            break
    
    
    
    print('\t\033[1mITERATION K-FOLD {}\033[0m \nIndexes of Train-Folds: {}, \033[1m Rows (Test Fold) to encode: {}\033[0m \n'.format(count, tr_ind, colored(val_ind, 'red')))  #sono gli indici delle folds
    
    X_tr, X_val = train.iloc[tr_ind], train.iloc[val_ind]
    
    diz_mean= X_tr.groupby(column_to_encode)[target_column].mean().to_dict()
    diz_std= X_tr.groupby(column_to_encode)[target_column].std().to_dict()
    
    #### Tratto come Nan chi non supera la threshold, mentre per gli altri faccio Encoding classico
    frequenze= X_tr.groupby('letter').size()
    cat_oltre_threshold= frequenze[frequenze >= threshold].index
    print(cat_oltre_threshold)
    
    train.loc[train.index[val_ind], name_encoded_mean] = X_val[column_to_encode].apply(lambda x: diz_mean[x] if x in cat_oltre_threshold else np.nan )
    train.loc[train.index[val_ind], name_encoded_std] = X_val[column_to_encode].apply(lambda x: diz_std[x] if x in cat_oltre_threshold else np.nan)
    
    
    mean_of_target = X_tr[target_column].mean()  #media di tutto il target sul KFold Train, not grouped
    std_of_target= X_tr[target_column].std()    # # std di tutto il target sul KFold Train, not grouped
    
    if show_steps== True:
        print("\033[1m mean and std of Target on K-1 folds are {} and {} \033[0m".format(mean_of_target, std_of_target))
        print('\033[1m', colored(train, 'blue'))
    
    #train[name_encoded_mean].fillna(mean_of_target, inplace = True) #ai NaN metto la media del KFold Train Target
    #train[name_encoded_std].fillna(std_of_target, inplace = True)   # ai NaN metto la std dev media del Kfold Train Target
    
    count+=1
    
train

/Users/federicofrancone/Library/Python/3.7/lib/python/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


	INITIAL TRAIN 
   letter  label
0      A      1
1      B      4
2      A      2
3      B      0
4      A      0
5      C      6 
 
 Threshold scelta: 0
Continue loop? y/n 
y
	ITERATION K-FOLD 1 
Indexes of Train-Folds: [2 3 4 5],  Rows (Test Fold) to encode: [0 1] 

Index(['A', 'B', 'C'], dtype='object', name='letter')
 mean and std of Target on K-1 folds are 2.0 and 2.8284271247461903 
   letter  label  letter_encoded_mean  letter_encoded_std
0      A      1                  1.0            1.414214
1      B      4                  0.0                 NaN
2      A      2                  NaN                 NaN
3      B      0                  NaN                 NaN
4      A      0                  NaN                 NaN
5      C      6                  NaN                 NaN
Continue loop? y/n 
y
	ITERATION K-FOLD 2 
Indexes of Train-Folds: [0 1 4 5],  Rows (Test Fold) to encode: [2 3] 

Index(['A', 'B', 'C'], dtype='object', name='letter')
 mean and std of Target on K-1 folds are

,letter,label,letter_encoded_mean,letter_encoded_std
0,A,1,1.0,1.414214
1,B,4,0.0,NaN
2,A,2,0.5,0.707107
3,B,0,4.0,NaN
4,A,0,1.5,0.707107
5,C,6,NaN,NaN


In [6]:
test

,letter,label
0,A,10
1,D,8


## Same as before, but adding the Imputation of Missing Values (not only NaNs, but also the very less frequent classes) respectively with mean and std of the Target for the whole population (the whole categorical column)

In [8]:
f= train.groupby('letter').size() 
f[f> 1]

letter
A    3
B    2
dtype: int64

In [5]:
kf = KFold(n_splits = 3, shuffle = False, random_state= 0)

name_encoded_mean= train.columns[0] + '_encoded_mean'
name_encoded_std= train.columns[0] + '_encoded_std'

column_to_encode= 'letter'
target_column= 'label'

threshold= 1
count= 1


for tr_ind, val_ind in kf.split(train):
    print('\tITERATION K-FOLD {} \nIndici delle Train-Folds: {}, \033[1mTest Fold da fillare: {}\033[0m \n'.format(count, tr_ind, val_ind))  #sono gli indici delle folds
    
    X_tr, X_val = train.iloc[tr_ind], train.iloc[val_ind]
    
    diz_mean= X_tr.groupby(column_to_encode)[target_column].mean().to_dict()
    diz_std= X_tr.groupby(column_to_encode)[target_column].std().to_dict()
    
     #### I consider as NaN the classes having frequency lower than a thresholds
    frequenze= X_tr.groupby('letter').size()
    cat_oltre_threshold= frequenze[frequenze >= threshold].index
    print(cat_oltre_threshold)
    
    train.loc[train.index[val_ind], name_encoded_mean] = X_val[column_to_encode].apply(lambda x: diz_mean[x] if x in cat_oltre_threshold else np.nan )
    train.loc[train.index[val_ind], name_encoded_std] = X_val[column_to_encode].apply(lambda x: diz_std[x] if x in cat_oltre_threshold else np.nan)
    
    
    mean_of_target = X_tr[target_column].mean()  #media di tutto il target sul KFold Train, not grouped
    std_of_target= X_tr[target_column].std()    # # std di tutto il target sul KFold Train, not grouped
    
    
    train[name_encoded_mean].fillna(mean_of_target, inplace = True) #ai NaN metto la media del KFold Train Target
    train[name_encoded_std].fillna(std_of_target, inplace = True)   # ai NaN metto la std dev media del Kfold Train Target
    
    count+=1
    
train

	ITERATION K-FOLD 1 
Indici delle Train-Folds: [2 3 4 5], Test Fold da fillare: [0 1] 

Index(['A', 'B', 'C'], dtype='object', name='letter')
	ITERATION K-FOLD 2 
Indici delle Train-Folds: [0 1 4 5], Test Fold da fillare: [2 3] 

Index(['A', 'B', 'C'], dtype='object', name='letter')
	ITERATION K-FOLD 3 
Indici delle Train-Folds: [0 1 2 3], Test Fold da fillare: [4 5] 

Index(['A', 'B'], dtype='object', name='letter')


/Users/federicofrancone/Library/Python/3.7/lib/python/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


,letter,label,letter_encoded_mean,letter_encoded_std
0,A,1,1.00,1.414214
1,B,4,0.00,2.828427
2,A,2,0.50,0.707107
3,B,0,4.00,2.753785
4,A,0,1.50,0.707107
5,C,6,1.75,1.707825


In [6]:
train.iloc[:4]['label'].describe().head(3)

count    4.000000
mean     1.750000
std      1.707825
Name: label, dtype: float64

## Encoding on Test Set, dealing with unseen categories

In [7]:
test

,letter,label
0,A,10
1,D,8


In [8]:
train.groupby('letter').mean()

,label,letter_encoded_mean,letter_encoded_std
letter,,,
A,1,1.00,0.942809
B,2,2.00,2.791106
C,6,1.75,1.707825


In [10]:
test[name_encoded_mean]= test['letter'].map(train.groupby('letter').mean()['letter_encoded_mean']) #mapping categories with their mean on Train Set
test[name_encoded_std]= test['letter'].map(train.groupby('letter').mean()['letter_encoded_std']) #mapping categories with their mean on Test
test

,letter,label,letter_encoded_mean,letter_encoded_std
0,A,10,1.0,0.942809
1,D,8,NaN,NaN


## Filling mean e std dev of new categories in Test Set with the respective means of in the Train

In [11]:
train

,letter,label,letter_encoded_mean,letter_encoded_std
0,A,1,1.00,1.414214
1,B,4,0.00,2.828427
2,A,2,0.50,0.707107
3,B,0,4.00,2.753785
4,A,0,1.50,0.707107
5,C,6,1.75,1.707825


In [12]:
mean_letter_encoded_tr= train['letter_encoded_mean'].mean()
std_letter_encoded_tr= train['letter_encoded_std'].mean()

test['letter_encoded_mean'].fillna(mean_letter_encoded_tr, inplace = True) #ai NaN metto la media di encoding_mean
test['letter_encoded_std'].fillna(std_letter_encoded_tr, inplace = True)   #ai NaN metto la media di encoding_std

test

,letter,label,letter_encoded_mean,letter_encoded_std
0,A,10,1.000000,0.942809
1,D,8,1.458333,1.686411


# Final Step, adding to Train and Test some Noise, which is proportional for each category to its standard deviation on the target.
<h4> The new variable will be a random drawing from a Gaussian centered into the Mean_encoding and with std dev equal to percentage_noise*Std_encdoing, where percentage_noise might be reasonable to be around 0.1 </h6>

In [14]:
np.random.seed(42)  #fisso un random seed
np.random.normal(1, 0.1*1.414)

1.0702353812357883

In [15]:
np.random.seed(42)  #fisso un random seed

percentage_noise= 0.01

train['ENCODED WITH NOISE']= np.random.normal(train['letter_encoded_mean'], percentage_noise*train['letter_encoded_std'],
                                              size= len(train))

train

,letter,label,letter_encoded_mean,letter_encoded_std,ENCODED WITH NOISE
0,A,1,1.00,1.414214,1.007025
1,B,4,0.00,2.828427,-0.003911
2,A,2,0.50,0.707107,0.504580
3,B,0,4.00,2.753785,4.041941
4,A,0,1.50,0.707107,1.498344
5,C,6,1.75,1.707825,1.746001


In [18]:
### On Test Set

test['ENCODED WITH NOISE']= np.random.normal(test['letter_encoded_mean'], percentage_noise*test['letter_encoded_std'], size= len(test))

test

,letter,label,letter_encoded_mean,letter_encoded_std,ENCODED WITH NOISE
0,A,10,1.000000,0.942809,0.995631
1,D,8,1.458333,1.686411,1.450479


# Generalizing and automating putting everything into a single class

### <font color= 'green'> <center> K-Fold Target Encoding with Noise

### Macro Explanation  <a id='kfold_targ_enc'> <br>
1. First, read this article to understand why and how a K-Fold approach is preferred to a basic target encoding (where each category is replaced with the related target mean of the training set) https://medium.com/@pouryaayria/k-fold-target-encoding-dfe9a594874b . It must be clear that this way overfitting is strongly reduced, since the same category will not be converted always with the same value, but a different one for each fold.
2. In past projects, I improved this approach, basically by adding:
    1. **Noise** to each value, which is proportional to the std deviation of the target of a given category     (considered in the K-1 folds..). Reason: suppose that category A has got values 0.3, 0.6 and 0.9, with mean= 0.6. Category B has 0.55, 0.60 and 0.65, with mean 0.6 again. It is obvious that the second one has got a much stronger dependency with the target, in respect to the first. This information is captured by the std dev: category A will be replaced in each row with 0.6 plus a noise which will be proportional to the standard deviation (my default multiplying factor is 0.1* std deviation). 
The conclusion is that the same category will have different values in each row, rather than in each fold: that's great to avoid overfitting!
    2. **Threshold**: suppose that in the training data you have 1000 observations of category A and only 5 observations having category B. Clearly, the target related to the latter is much more uncertain: actually we don't want any model to learn the relationship between category B and the target: if it does, it would be likely to overfit. Thus, the K-Fold targ. encoding approach is applied only for those categories having a number of samples (in the K-1 folds) higher than a chosen threshold. If this condition is not satisfied, they are replaced with the mean/median of the whole feature in the training data (similar to a binning approach: by filling all the low frequency category with the mean, you are actually grouping them into a common cluster..) <br>    
3. In the test set, each category is replaced with its corresponding mean in the train. There is no need to add noise here, since it has already been added in the train. If there is a new category - unseen in the train data - it is filled with the training mean of the whole feature (similar to a missing values imputation..)
    
4. <font color ='green'> I will provide a jupyter notebook (I have to clean the current one I have) with a step by step grapchical explanation of the whole process. I also have to clean the function and comments, translating to english italian comments

<b> Referring to the previous description, the main parameters of the function belowe are: </b>
1. **n_folds**: is the number of Folds it is wanted to use. Very simply, a higher number prevents even more overfitting, but it is computationally more expensive. I usually use 100 folds, but I don't advice to consider this as an hyperparameter of the model.
2. **prop_std_per_noise**: it is the mupltiplying factor of the std deviation, it has to be a low number, otherwise you are adding to much noise to the data and you lose too much information. Typically, I choose values lower than 0.1. Play with the parameter in the cell below to have an idea of the impact this multiplying factor has.
3. **threshold**: see cell above.
4. **cols_to_encode**: by default the function performs this encoding for all the categorical features (that is, being 'object' type). But if you want to apply only for some specific columns, you can pass them as a list. <br>
Less Relevant:
5. *target*: is simply the target name
6. *minimum_std*: for some categories it can happen that the std deviation is 0. To reduce the power of this (and overfitting..), we set a minimum amount of std dev, without which there would not be any noise added. Thus, set this value low, I usually use smth like 0.05

In [19]:
class KFoldTargetEncoderTrain_std(base.BaseEstimator, base.TransformerMixin):
    def __init__(self, colnames,targetName, n_fold=10, verbosity=True, 
                 discardOriginal_col=False, threshold= 1, showPrint= False):

        self.colnames = colnames
        self.targetName = targetName
        self.n_fold = n_fold
        self.verbosity = verbosity
        self.discardOriginal_col = discardOriginal_col
        self.threshold= threshold
        self.showPrint= showPrint

    def fit(self, X, y=None):
        return self

    def transform(self,X):

        assert(type(self.targetName) == str)
        assert(type(self.colnames) == str)
        assert(self.colnames in X.columns)
        assert(self.targetName in X.columns)
        assert(type(self.threshold) == int)
       
        kf = KFold(n_splits = self.n_fold, shuffle = False, random_state=2019)
        #kf = StratifiedKFold(n_splits = self.n_fold, shuffle = False, random_state=2019)

        col_mean_name = self.colnames + '_' + 'enc_mean'
        col_std_name= self.colnames + '_' + 'enc_std'
        
        #Le inizializzo con tutti NaN
        X[col_mean_name] = np.nan
        X[col_std_name] = np.nan
        
        #A ogni giro (per ogni K-Fold) fillo valori della Test Fold con la media delle Train Fold.
        for tr_ind, val_ind in kf.split(X):
            if self.showPrint is True:
                print('Indici delle Train-Folds: {},\033[1mTest Fold da fillare: {}\033[0m \n'.format(tr_ind, val_ind))  #sono gli indici delle folds

            X_tr, X_val = X.iloc[tr_ind], X.iloc[val_ind]

            diz_mean= X_tr.groupby(self.colnames)[self.targetName].mean().to_dict()
            diz_std= X_tr.groupby(self.colnames)[self.targetName].std().to_dict()

            #### Tratto come Nan chi non supera la threshold, agli altri assegno media e std sulle altre folds
            frequenze= X_tr.groupby(self.colnames).size()
            cat_oltre_threshold= frequenze[frequenze >= self.threshold].index
            
            if self.showPrint == True:
                print('cat_oltre_threshold', cat_oltre_threshold)

            X.loc[X.index[val_ind], col_mean_name] = X_val[self.colnames].apply(lambda x: diz_mean[x] if x in cat_oltre_threshold else np.nan )
            X.loc[X.index[val_ind], col_std_name] = X_val[self.colnames].apply(lambda x: diz_std[x] if x in cat_oltre_threshold else np.nan)

            mean_of_target = X_tr[self.targetName].mean()  #media di tutto il target sul KFold Train, not grouped
            std_of_target= X_tr[self.targetName].std()    # # std di tutto il target sul KFold Train, not grouped

            X[col_mean_name].fillna(mean_of_target, inplace = True) #ai NaN metto la media del KFold Train Target
            X[col_std_name].fillna(std_of_target, inplace = True)   # ai NaN metto la std dev media del Kfold Train Target

        if self.verbosity:

            encoded_feature = X[col_mean_name].values
            print('Correlation between the new feature, {} and, {} is {}.'.format(col_mean_name,
                                    self.targetName,np.corrcoef(X[self.targetName].values, encoded_feature)[0][1]))
            
        if self.discardOriginal_col:
            if self.showPrint == True:
                print('columns: ', X.columns)
            X = X.drop(self.colnames, axis=1)
        return X
    
    
class KFoldTargetEncoderTest_std(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self,train,colNames,Train_encoded_mean, Train_encoded_std, showPrint= False):
        
        self.train = train
        self.colNames = colNames
        self.Train_encoded_mean = Train_encoded_mean
        self.Train_encoded_std = Train_encoded_std
        self.showPrint= showPrint
        
    def fit(self, X, y=None):
        return self

    def transform(self,X):

        ### Media e std del Target grouped per categoria su tutto il Train
        category_mean_tr = self.train[[self.colNames,self.Train_encoded_mean]].groupby(self.colNames).mean() #.reset_index() 
        category_std_tr= self.train[[self.colNames,self.Train_encoded_std]].groupby(self.colNames).mean() #.reset_index() 
        
        ### Media dell'Encoding basato su mean e std sul Train (non grouped per categoria)
        population_mean_tr= self.train[self.Train_encoded_mean].mean()
        population_std_tr= self.train[self.Train_encoded_std].mean()
        
        ### Metto nel Test i valori medi della mean e std del Train per quella categoria
        X[self.Train_encoded_mean]= X[self.colNames].map(category_mean_tr.squeeze().to_dict())
        X[self.Train_encoded_std]= X[self.colNames].map(category_std_tr.squeeze().to_dict())
        
        ### Fillo i Missing Values (Categorie Nuove) con i valori medi di mean e std (encodate) di tutte la variabile nel Train
        if self.showPrint== True:
            print('Filled {} Missing Values with Average mean and std equals to {}, {}'.format(
                                                        np.sum(np.sum(X.isna())), population_mean_tr , population_std_tr))
        
        X[self.Train_encoded_mean].fillna(population_mean_tr, inplace = True) #fillo con media di enc_mean sul Train
        X[self.Train_encoded_std].fillna(population_std_tr, inplace = True)   #fillo con media di enc_std sul Train
        
        return X

### Let's verify that using the class I obtain the same results of the experimental procedure above

In [20]:
########################### Creo Training Set ##############################
copy_train= pd.DataFrame()

tr_letters= 'A B A B A C'.split()
tr_labels= [1, 4, 2, 0, 0, 6]

copy_train['letter']= tr_letters
copy_train['label']= tr_labels


################ Creo Test: ci metto una categoria nota, A, e un'altra non vista nel Train, D ########################
test_letters= 'A D'.split()
test_labels= [10, 8]

copy_test= pd.DataFrame()
copy_test['letter']= test_letters
copy_test['label']= test_labels

copy_train

,letter,label
0,A,1
1,B,4
2,A,2
3,B,0
4,A,0
5,C,6


In [22]:
targetc = KFoldTargetEncoderTrain_std('letter', 'label' , n_fold= 3)
copy_train_enc = targetc.fit_transform(copy_train)
copy_train_enc

Correlation between the new feature, letter_enc_mean and, label is -0.3841614968388928.


/Users/federicofrancone/Library/Python/3.7/lib/python/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


,letter,label,letter_enc_mean,letter_enc_std
0,A,1,1.00,1.414214
1,B,4,0.00,2.828427
2,A,2,0.50,0.707107
3,B,0,4.00,2.753785
4,A,0,1.50,0.707107
5,C,6,1.75,1.707825


In [24]:
### Old train which I obtained without using the class
train

,letter,label,letter_encoded_mean,letter_encoded_std,ENCODED WITH NOISE
0,A,1,1.00,1.414214,1.007025
1,B,4,0.00,2.828427,-0.003911
2,A,2,0.50,0.707107,0.504580
3,B,0,4.00,2.753785,4.041941
4,A,0,1.50,0.707107,1.498344
5,C,6,1.75,1.707825,1.746001


### Correct; I obtained the same I had before. Now let's verify for the Test Set

In [25]:
copy_train_enc.mean()

label              2.166667
letter_enc_mean    1.458333
letter_enc_std     1.686411
dtype: float64

In [26]:
test_targetc = KFoldTargetEncoderTest_std(copy_train_enc, 'letter', 'letter_enc_mean', 'letter_enc_std')
copy_test_enc= test_targetc.fit_transform(copy_test)
copy_test_enc

,letter,label,letter_enc_mean,letter_enc_std
0,A,10,1.000000,0.942809
1,D,8,1.458333,1.686411


In [27]:
test

,letter,label,letter_encoded_mean,letter_encoded_std,ENCODED WITH NOISE
0,A,10,1.000000,0.942809,0.995631
1,D,8,1.458333,1.686411,1.450479


### Correct, I obtained the same values!

### NOTE: Where I have std dev = 0 for a certain category, that is because in the K-fold Train I have all same values for such category. In that case I would have no noise; thus in the official method I put a default std dev (around 0.01) in all such cases. This is, as the main purpose of the whole process, to reduce overfitting.